In [1]:
!pip install beyondllm

  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached openai-1.20.0-py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-1.10.18-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (152 kB)
  Using cached llama_index_core-0.10.68.post1-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_llms_openai-0.1.31-py3-none-any.whl.metadata (650 bytes)
INFO: pip is looking at multiple versions of llama-index-llms-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached llama_index_llms_openai-0.1.30-py3-none-any.whl.metadata (650 bytes)
  Using cached llama_index_llms_openai-0.1.29-py3-none-any.whl.metadata (650 bytes)
  Using cached llama_index_llms_openai-0.1.28-py3-none-any.whl.metadata (650 bytes)
  Using cached llama_index_llms_openai-0.1.27-py3-none-any.whl.metadata (610 bytes)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
Using cached openai-1.20.0-py3-none-any.whl (292 kB)
Usi

In [2]:
!pip install llama-index-finetuning
!pip install llama-index-embeddings-huggingface

  Using cached llama_index_core-0.11.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached llama_index_llms_openai-0.2.0-py3-none-any.whl.metadata (648 bytes)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached openai-1.42.0-py3-none-any.whl.metadata (22 kB)
Using cached llama_index_core-0.11.1-py3-none-any.whl (1.6 MB)
Using cached llama_index_llms_openai-0.2.0-py3-none-any.whl (12 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached openai-1.42.0-py3-none-any.whl (362 kB)
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.18
    Uninstalling pydantic-1.10.18:
      Successfully uninstalled pydantic-1.10.18
  Attempting uninstall: openai
    Found existing instal

## 1. Import Required Libraries

First, import the necessary libraries and set up the environment.

In [3]:
from beyondllm import source, retrieve, llms,generator
from beyondllm.embeddings import FineTuneEmbeddings
import os

In [4]:
from getpass import getpass

os.environ['GOOGLE_API_KEY'] = getpass("API key:")

API key:··········


2. Load and Prepare Data

Load your data from a specified path and prepare it for processing. The data should be in PDF format for this example, and it will be chunked appropriately.

In [5]:
path = "/content/resume_ds.pdf"

In [6]:
data = source.fit(path, dtype="pdf", chunk_size=1024, chunk_overlap=0)

### 3. Initialize the Language Model

We are using the GeminiModel.

In [7]:
llm = llms.GeminiModel()

### 4. Fine-Tune the Embeddings

The model used in this case is the BAAI/bge-small-en-v1.5, an open-source embedding model from HuggingFace.

In [8]:
fine_tuned_model = FineTuneEmbeddings()

In [9]:
embed_model = fine_tuned_model.train([path], "BAAI/bge-small-en-v1.5", llm, "fintune")

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsed 1 nodes


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsed 1 nodes


100%|██████████| 1/1 [00:05<00:00,  5.39s/it]


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

> Note: In the training function, we declare a path where the model needs to be saved. For our case, this path is set to finetune. This same path variable is also used to load the fine-tuned embedding model.


## 5. Load the Fine-Tuned Model

HuggingFace embeddings now are updated, so we will now use that in our retrieval and generation pipeline.

In [10]:
embed_model = fine_tuned_model.load_model("fintune")

## 6. Set Up the Retriever

Set up the retriever using the fine-tuned embeddings. The retriever is responsible for fetching relevant documents based on the query.

In [11]:
retriever = retrieve.auto_retriever(data, embed_model, type="normal", top_k=4)

## 7. Retrieve Information

You can now use the retriever to fetch relevant information. For example, to retrieve information about "Tarun's role at AI Planet":

In [12]:
retriever.retrieve("what is Esther role in KAA")[0].text

'ESTHER  NELIMA   \nDATA SCIENTIST  \nE-mail Address     LinkedIn Profile     GitHub Profile  \nPhone: +254 741 920 648  \n \nMotivated fresher with degree in Mathematics, pursuing certification in Data Science. Skilled in \nbuilding, optimizing and maintaining ML applications. Able to derive actionable insights from data.  \n \nSkills :   \n \no Python:  Numpy, Pandas, Matplotlib, Seaborn, TensorFlow, Keras, Time, Scikit -Learn, SciPy, ArcPy, \nGeopandas, Beautiful Soup  \no SQL:  BigQuery, MySql  \no Collaboration:  Git, Github  \no Mathematics:  Probability, Regression, Linear algebra, Real Analysis, Numerical Analysis.  \no Data Analytics : Data Preprocessing, Exploration, Visualization, Storytelling  \no Machine Learning: Anomaly detection, Forecasting, GIS (Familiar), Recommender Systems and \nCustomer Segmentation.  \no Data Management:  Hadoop  (Big Data)   \no Cloud computing: AWS, Google Cloud  \no Data Visualization: Power BI, Tableau.  \no Spreadsheets: MS Excel, Google She

## 8. Generate Responses with RAG & Evaluate Performance


In [14]:
pipeline = generator.Generate(question="Esther's role at KAA summarize it",retriever=retriever,llm=llm)

In [15]:
print(pipeline.call())

Esther worked as a Monitoring and Evaluation Trainee at Kenya Airports Authority – JKIA, Nairobi from OCT 2021 - DEC 2021. During her time there, she analyzed project status and generated dashboards for progress monitoring. She also contributed to research and development initiatives, including questionnaire coding and data entry.


In [16]:
print(pipeline.get_rag_triad_evals())

Executing RAG Triad Evaluations...
Context relevancy Score: 4.5
This response does not meet the evaluation threshold. Consider refining the structure and content for better clarity and effectiveness.
Answer relevancy Score: 8.0
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.
Groundness score: 8.0
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.
